
# 5. split wgs data

i need to split my data so that i can calculate 

LD

nucleotide diversity

theta

- cca

- sbpa

- by chromosome

I will split the SNP data and the wgs data


## SNP

note:

i moved all data to /Volumes/exharddisk/

so instead of 

Users/josephinelovemachine/soyadapt_data_analysis_with_start_data/start_data 

its 

/Volumes/exharddisk/soyadapt_data_analysis_with_start_data/start_data 

this is the case for all the data
and also where all the /Volumes/exharddisk/soyadapt_data_analysis_with_start_data/generated_data/
will be stored. 


@ /Volumes/exharddisk/soyadapt_data_analysis_with_start_data/start_data

here i have a copy of the file

intersect_filtered.vcf.gz

i renamed the file snp.vcf.gz for my own convenience:

bash:

cp intersect_filtered.vcf.gz /Volumes/exharddisk/soyadapt_data_analysis_with_start_data/start_data/snp.vcf.gz

i have a list of the accession names in a .txt

cca.txt
sbpa.txt 


I am at this point in my analysis i am removing the accessions 006A and 007A because they do not seem to be part of the specific sbp i am looking at although they are swedish accessions.  Therefore these accessions are excluded from the data set for the diversity analysis (H2).

I am also not including the "founders" in this nex part of the analysis.

snp data in a file with only the cca accessions:

vcftools --gzvcf snp.vcf.gz --keep cca.txt --out snp_cca --recode -c | bgzip -c > snp_cca.vcf.gz  #the log is called snp_cca.log
bcftools index snp_cca.vcf.gz

#the same goes for sbpa
vcftools --gzvcf snp.vcf.gz --keep sbpa.txt --out snp_sbpa --recode -c | bgzip -c > snp_sbpa.vcf.gz
bcftools index snp_sbpa.vcf.gz 

check
bcftools view --no-header -G -m 2 -M 2 --types snps snp.vcf.gz | wc -l
35486
bcftools view --no-header -G -m 2 -M 2 --types snps snp_cca.vcf.gz | wc -l
35486
bcftools view --no-header -G -m 2 -M 2 --types snps snp_sbpa.vcf.gz | wc -l
35486
bcftools query -l snp_cca.vcf.gz | wc -l
409 and 153 for sbpa


then i will split the vcf file by chromosome so i can calculate LD/chromosome
cca_1.vcf.gz
cca_2.vcf.gz etc.

* here!
bcftools view snp_cca.vcf.gz -r 1 -Oz -o cca_1.vcf.gz



I used:
bcftools view snp_cca.vcf.gz -r 1 -Oz -o snp_cca_1.vcf.gz |bcftools view snp_cca.vcf.gz -r 2 -Oz -o snp_cca_2.vcf.gz | bcftools view snp_cca.vcf.gz -r 3 -Oz -o snp_cca_3.vcf.gz | bcftools view snp_cca.vcf.gz -r 4 -Oz -o snp_cca_4.vcf.gz | bcftools view snp_cca.vcf.gz -r 5 -Oz -o snp_cca_5.vcf.gz | bcftools view snp_cca.vcf.gz -r 6 -Oz -o snp_cca_6.vcf.gz | bcftools view snp_cca.vcf.gz -r 7 -Oz -o snp_cca_7.vcf.gz | bcftools view snp_cca.vcf.gz -r 8 -Oz -o snp_cca_8.vcf.gz | bcftools view snp_cca.vcf.gz -r 9 -Oz -o snp_cca_9.vcf.gz | bcftools view snp_cca.vcf.gz -r 10 -Oz -o snp_cca_10.vcf.gz | bcftools view snp_cca.vcf.gz -r 11 -Oz -o snp_cca_11.vcf.gz | bcftools view snp_cca.vcf.gz -r 12 -Oz -o snp_cca_12.vcf.gz | bcftools view snp_cca.vcf.gz -r 13 -Oz -o snp_cca_13.vcf.gz | bcftools view snp_cca.vcf.gz -r 14 -Oz -o snp_cca_14.vcf.gz | bcftools view snp_cca.vcf.gz -r 15 -Oz -o snp_cca_15.vcf.gz | bcftools view snp_cca.vcf.gz -r 16 -Oz -o snp_cca_16.vcf.gz | bcftools view snp_cca.vcf.gz -r 17 -Oz -o snp_cca_17.vcf.gz | bcftools view snp_cca.vcf.gz -r 18 -Oz -o snp_cca_18.vcf.gz | bcftools view snp_cca.vcf.gz -r 19 -Oz -o snp_cca_19.vcf.gz | bcftools view snp_cca.vcf.gz -r 20 -Oz -o snp_cca_20.vcf.gz 

check
bcftools query -l snp_cca_1.vcf.gz | wc -l
409
bcftools view --no-header -G -m 2 -M 2 --types snps snp_cca_1.vcf.gz | wc -l


split snp_sbpa up in chr too

bcftools view snp_sbpa.vcf.gz -r 1 -Oz -o snp_sbpa_1.vcf.gz |bcftools view snp_sbpa.vcf.gz -r 2 -Oz -o snp_sbpa_2.vcf.gz | bcftools view snp_sbpa.vcf.gz -r 3 -Oz -o snp_sbpa_3.vcf.gz | bcftools view snp_sbpa.vcf.gz -r 4 -Oz -o snp_sbpa_4.vcf.gz | bcftools view snp_sbpa.vcf.gz -r 5 -Oz -o snp_sbpa_5.vcf.gz | bcftools view snp_sbpa.vcf.gz -r 6 -Oz -o snp_sbpa_6.vcf.gz | bcftools view snp_sbpa.vcf.gz -r 7 -Oz -o snp_sbpa_7.vcf.gz | bcftools view snp_sbpa.vcf.gz -r 8 -Oz -o snp_sbpa_8.vcf.gz | bcftools view snp_sbpa.vcf.gz -r 9 -Oz -o snp_sbpa_9.vcf.gz | bcftools view snp_sbpa.vcf.gz -r 10 -Oz -o snp_sbpa_10.vcf.gz | bcftools view snp_sbpa.vcf.gz -r 11 -Oz -o snp_sbpa_11.vcf.gz | bcftools view snp_sbpa.vcf.gz -r 12 -Oz -o snp_sbpa_12.vcf.gz | bcftools view snp_sbpa.vcf.gz -r 13 -Oz -o snp_sbpa_13.vcf.gz | bcftools view snp_sbpa.vcf.gz -r 14 -Oz -o snp_sbpa_14.vcf.gz | bcftools view snp_sbpa.vcf.gz -r 15 -Oz -o snp_sbpa_15.vcf.gz | bcftools view snp_sbpa.vcf.gz -r 16 -Oz -o snp_sbpa_16.vcf.gz | bcftools view snp_sbpa.vcf.gz -r 17 -Oz -o snp_sbpa_17.vcf.gz | bcftools view snp_sbpa.vcf.gz -r 18 -Oz -o snp_sbpa_18.vcf.gz | bcftools view snp_sbpa.vcf.gz -r 19 -Oz -o snp_sbpa_19.vcf.gz | bcftools view snp_sbpa.vcf.gz -r 20 -Oz -o snp_sbpa_20.vcf.gz

bcftools query -l snp_sbpa.vcf.gz | wc -l
153




#count the snps per chromosome in snp_CCA:
bcftools view --no-header -G -m 2 -M 2 --types snps snp_cca_1.vcf.gz | wc -l > /Users/josephinelovemachine/soyadapt_data_analysis_with_start_data/resources/generated_data/snp_cca_snp_counts_per_chr.txt

for x in {1..20}
do
bcftools view --no-header -G -m 2 -M 2 --types snps snp_cca_$x.vcf.gz | wc -l >> /Users/josephinelovemachine/soyadapt_data_analysis_with_start_data/resources/generated_data/snp_cca_snp_counts_per_chr.txt 
done

snp_SBPA
for x in {1..20}
do
bcftools view --no-header -G -m 2 -M 2 --types snps snp_sbpa_$x.vcf.gz | wc -l >> /Users/josephinelovemachine/soyadapt_data_analysis_with_start_data/resources/generated_data/snp_sbpa_snp_counts_per_chr.txt 
done

bcftools view --no-header -G -m 2 -M 2 --types snps -r  snp_sbpa.vcf.gz | wc -l




check
bcftools query -l snp_cca.vcf.gz | wc -l
409
bcftools query -l snp_sbpa.vcf.gz | wc -l
153
bcftools view --no-header -G -m 2 -M 2 --types snps snp_cca.vcf.gz | wc -l
8533444
bcftools view --no-header -G -m 2 -M 2 --types snps snp_sbpa.vcf.gz | wc -l
8533444
bcftools view --no-header -G -m 2 -M 2 --types snps snp_cca_snp_sbpa_filtered.vcf.gz | wc -l

bcftools query -f'[%CHROM]\n' snp_cca.vcf.gz | uniq -c
1426 
2070 
1414 
1730 
1551 
1657 
1818 
2185 
1654 
1904 
1409 
1416 
2277 
1630 
2079 
1461 
1721 
2786 
1950 
1348
bcftools query -f'[%CHROM]\n' wgs.vcf.gz | uniq -l

## WGS

at /Users/josephinelovemachine/soyadapt_data_analysis_with_start_data/resources/start_data

here i will copy the file and give it a new name:
bash:
cp cca_sbpa_filtered.vcf.gz wgs.vcf.gz
bcftools index wgs.vcf.gz

i will generate a list of the accession names in a .txt

cca.txt
sbpa.txt 
done

I am at this point in my analysis i am removing the accessions 006A and 007A because they do not seem to be part of the specific sbp i am looking at although they are swedish accessions.  Therefore these accessions are excluded from the data set for the diversity analysis (H2)


In [ ]:
vcftools --gzvcf wgs.vcf.gz --keep cca.txt --out cca --recode -c | bgzip -c > cca.vcf.gz  #the log is called cca.log
bcftools index cca.vcf.gz

#the same goes for sbpa
vcftools --gzvcf wgs.vcf.gz --keep sbpa.txt --out sbpa --recode -c | bgzip -c > sbpa.vcf.gz
bcftools index sbpa.vcf.gz 
done
check
bcftools view --no-header -G -m 2 -M 2 --types snps wgs.vcf.gz | wc -l
8533444
bcftools view --no-header -G -m 2 -M 2 --types snps cca.vcf.gz | wc -l
8533444
bcftools view --no-header -G -m 2 -M 2 --types snps sbpa.vcf.gz | wc -l
8533444
bcftools query -l cca.vcf.gz | wc -l
409


then i will split the vcf file by chromosome so i can calculate LD/chromosome
cca_1.vcf.gz
cca_2.vcf.gz etc.

bcftools view cca.vcf.gz -r 1 -Oz -o cca_1.vcf.gz

I used:
bcftools view cca.vcf.gz -r 1 -Oz -o cca_1.vcf.gz |bcftools view cca.vcf.gz -r 2 -Oz -o cca_2.vcf.gz | bcftools view cca.vcf.gz -r 3 -Oz -o cca_3.vcf.gz | bcftools view cca.vcf.gz -r 4 -Oz -o cca_4.vcf.gz | bcftools view cca.vcf.gz -r 5 -Oz -o cca_5.vcf.gz | bcftools view cca.vcf.gz -r 6 -Oz -o cca_6.vcf.gz | bcftools view cca.vcf.gz -r 7 -Oz -o cca_7.vcf.gz | bcftools view cca.vcf.gz -r 8 -Oz -o cca_8.vcf.gz | bcftools view cca.vcf.gz -r 9 -Oz -o cca_9.vcf.gz | bcftools view cca.vcf.gz -r 10 -Oz -o cca_10.vcf.gz | bcftools view cca.vcf.gz -r 11 -Oz -o cca_11.vcf.gz | bcftools view cca.vcf.gz -r 12 -Oz -o cca_12.vcf.gz | bcftools view cca.vcf.gz -r 13 -Oz -o cca_13.vcf.gz | bcftools view cca.vcf.gz -r 14 -Oz -o cca_14.vcf.gz | bcftools view cca.vcf.gz -r 15 -Oz -o cca_15.vcf.gz | bcftools view cca.vcf.gz -r 16 -Oz -o cca_16.vcf.gz | bcftools view cca.vcf.gz -r 17 -Oz -o cca_17.vcf.gz | bcftools view cca.vcf.gz -r 18 -Oz -o cca_18.vcf.gz | bcftools view cca.vcf.gz -r 19 -Oz -o cca_19.vcf.gz | bcftools view cca.vcf.gz -r 20 -Oz -o cca_20.vcf.gz 

check
bcftools query -l cca_1.vcf.gz | wc -l
409
bcftools view --no-header -G -m 2 -M 2 --types snps cca_1.vcf.gz | wc -l
464964

In [ ]:
# split sbpa up in chr too

bcftools view sbpa.vcf.gz -r 1 -Oz -o sbpa_1.vcf.gz |bcftools view sbpa.vcf.gz -r 2 -Oz -o sbpa_2.vcf.gz | bcftools view sbpa.vcf.gz -r 3 -Oz -o sbpa_3.vcf.gz | bcftools view sbpa.vcf.gz -r 4 -Oz -o sbpa_4.vcf.gz | bcftools view sbpa.vcf.gz -r 5 -Oz -o sbpa_5.vcf.gz | bcftools view sbpa.vcf.gz -r 6 -Oz -o sbpa_6.vcf.gz | bcftools view sbpa.vcf.gz -r 7 -Oz -o sbpa_7.vcf.gz | bcftools view sbpa.vcf.gz -r 8 -Oz -o sbpa_8.vcf.gz | bcftools view sbpa.vcf.gz -r 9 -Oz -o sbpa_9.vcf.gz | bcftools view sbpa.vcf.gz -r 10 -Oz -o sbpa_10.vcf.gz | bcftools view sbpa.vcf.gz -r 11 -Oz -o sbpa_11.vcf.gz | bcftools view sbpa.vcf.gz -r 12 -Oz -o sbpa_12.vcf.gz | bcftools view sbpa.vcf.gz -r 13 -Oz -o sbpa_13.vcf.gz | bcftools view sbpa.vcf.gz -r 14 -Oz -o sbpa_14.vcf.gz | bcftools view sbpa.vcf.gz -r 15 -Oz -o sbpa_15.vcf.gz | bcftools view sbpa.vcf.gz -r 16 -Oz -o sbpa_16.vcf.gz | bcftools view sbpa.vcf.gz -r 17 -Oz -o sbpa_17.vcf.gz | bcftools view sbpa.vcf.gz -r 18 -Oz -o sbpa_18.vcf.gz | bcftools view sbpa.vcf.gz -r 19 -Oz -o sbpa_19.vcf.gz | bcftools view sbpa.vcf.gz -r 20 -Oz -o sbpa_20.vcf.gz

bcftools query -l sbpa.vcf.gz | wc -l
153



#count the snps per chromosome in CCA:
bcftools view --no-header -G -m 2 -M 2 --types snps cca_1.vcf.gz | wc -l > /Users/josephinelovemachine/soyadapt_data_analysis_with_start_data/resources/generated_data/cca_snp_counts_per_chr.txt

for x in {1..20}
do
bcftools view --no-header -G -m 2 -M 2 --types snps cca_$x.vcf.gz | wc -l >> /Users/josephinelovemachine/soyadapt_data_analysis_with_start_data/resources/generated_data/cca_snp_counts_per_chr.txt 
done

SBPA
for x in {1..20}
do
bcftools view --no-header -G -m 2 -M 2 --types snps sbpa_$x.vcf.gz | wc -l >> /Users/josephinelovemachine/soyadapt_data_analysis_with_start_data/resources/generated_data/sbpa_snp_counts_per_chr.txt 
done

bcftools view --no-header -G -m 2 -M 2 --types snps -r  sbpa.vcf.gz | wc -l

chr1	464964
chr2	390047
chr3	444947
chr4	487558
chr5	289141
chr6	463901
chr7	366338
chr8	378479
chr9	427631
chr10	423082
chr11	248979
chr12	309852
chr13	363942
chr14	447452
chr15	537959
chr16	379570
chr17	364763
chr18	654166
chr19	466084
chr20	417623

check
bcftools query -l cca.vcf.gz | wc -l
409
bcftools query -l sbpa.vcf.gz | wc -l
153
bcftools view --no-header -G -m 2 -M 2 --types snps cca.vcf.gz | wc -l
8533444
bcftools view --no-header -G -m 2 -M 2 --types snps sbpa.vcf.gz | wc -l
8533444
bcftools view --no-header -G -m 2 -M 2 --types snps cca_sbpa_filtered.vcf.gz | wc -l

bcftools query -f'[%CHROM]\n' cca.vcf.gz | uniq -c

bcftools query -f'[%CHROM]\n' wgs.vcf.gz | uniq -l

## statistics

In [ ]:
Getting Fst population statistics

VCFtools can also calculate Fst statistics between individuals of different populations. It is an estimate calculated in accordance to Weir and Cockerham’s 1984 paper. The user must supply text files that contain lists of individuals (one per line) that are members of each population. The function will work with multiple populations if multiple --weir-fst-pop arguments are used. The following example shows how to calculate a per-site Fst calculation with two populations. Other arguments can be used in conjunction with this function, such as --fst-window-size and --fst-window-step. --fst-window-size 1000 and --fst-window-step 500

vcftools --gzvcf wgs.vcf.gz --weir-fst-pop sbpa.txt --weir-fst-pop cca.txt --fst-window-size 1000 --fst-window-step 500 --out sbpa_vs_cca

then i will use vcf tools to calculate nucleotide diversity

vcftools --vcf - --site-pi --positions SNP_list.txt --out nucleotide_diversity

then i will use vcftools to calculate theta

vcftools --vcf - --TajimaD 10000 --out theta

then i will use vcftools to calculate LD per chromosome 

vcftools --vcf - --hap-r2 --ld-window-bp 100000 --out LD

then i will use vcftools to calculate fst

vcftools --vcf - --weir-fst-pop cca.txt --weir-fst-pop sbpa.txt --out fst
    


from https://vcftools.sourceforge.net/documentation.html#freq

Getting allele frequency

To determine the frequency of each allele over all individuals in a VCF file, the --freq argument is used.

./vcftools --vcf input_data.vcf --freq --out output

The output file will be written to output.frq.
Getting sequencing depth information

Another useful output function summarizes sequencing depth for each individual or for each site. Just like the allele frequency example above, this output function follows the same basic model.

./vcftools --vcf input_data.vcf --depth -c > depth_summary.txt

With VCFtools, you can use many combinations of filters and an output function. For example, to write out site-wise sequence depths only at sites that have no missing data, include the --max-missing argument.

./vcftools --vcf input_data.vcf --site-depth --max-missing 1.0 --out site_depth_summary


Getting linkage disequilibrium statistics

Linkage disequilibrium between sites can be determined as well. This is accomplished using the --hap-r2, --geno-r2, or --geno-chisq arguments. Since the program must do pairwise site comparisons, this analysis can be time consuming, so it is recommended to filter the sites first or use one of the other options (--ld-window, --ld-window-bp or --min-r2) to reduce the number of comparisons. In this example, the VCFtools will only compare sites within 50,000 base pairs of one another.

./vcftools --vcf input_data.vcf --hap-r2 --ld-window-bp 50000 --out ld_window_50000


Getting Fst population statistics

VCFtools can also calculate Fst statistics between individuals of different populations. It is an estimate calculated in accordance to Weir and Cockerham’s 1984 paper. The user must supply text files that contain lists of individuals (one per line) that are members of each population. The function will work with multiple populations if multiple --weir-fst-pop arguments are used. The following example shows how to calculate a per-site Fst calculation with two populations. Other arguments can be used in conjunction with this function, such as --fst-window-size and --fst-window-step.

./vcftools --vcf input_data.vcf --weir-fst-pop population_1.txt --weir-fst-pop population_2.txt --out pop1_vs_pop2